In [16]:
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk

import matplotlib as mpl
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()

In [1]:
# csv 읽어들일 때 사용할 변수들 정의

# train.csv에서 dropoff_datetime 빼고 저장
cols_list = ['pickup_datetime', 'passenger_count', \
             'pickup_longitude', 'pickup_latitude', \
             'dropoff_longitude', 'dropoff_latitude',\
             'trip_duration']
# train.csv에서 dropoff_datetime 포함 저장
cols_list2 = ['pickup_datetime', 'dropoff_datetime', \
             'passenger_count', \
             'pickup_longitude', 'pickup_latitude', \
             'dropoff_longitude', 'dropoff_latitude', \
             'trip_duration']
# test.csv에서 저장
cols_test = ['pickup_datetime', 'passenger_count', \
             'pickup_longitude', 'pickup_latitude', \
             'dropoff_longitude', 'dropoff_latitude']

# cols_list 용
short_cols = ['p_dt', 'pas', 'p_lng', 'p_lat', 'd_lng', 'd_lat', 'dur']
# cols_list2 용
short_cols2 = ['p_dt', 'd_dt', 'pas', 'p_lng', 'p_lat', 'd_lng', 'd_lat', 'dur']
# cols_test 용
short_test = ['p_dt', 'pas', 'p_lng', 'p_lat', 'd_lng', 'd_lat']

In [2]:
# 경로 변수 (고정)

dataLoc = '../_data/'
trainfile = 'train.csv'
testfile = 'test.csv'

In [3]:
# train.csv
# dropoff_datetime을 뺄 경우: usecols=cols_list, .columns=short_cols
# dropoff_datetime을 넣을 경우: usecols=cols_list2, .columns=short_cols2

# train.csv (날짜 대충 읽는 버전, 빠름)
taxi_df = pd.read_csv(dataLoc + trainfile, usecols=cols_list)
taxi_df.columns = short_cols
taxi_df.head()

# train.csv (날짜 제대로 읽어오는 버전, 오래걸림)
# taxi_df = pd.read_csv(dataLoc + trainfile, \
#                       usecols=cols_list, \
#                       infer_datetime_format=True, \
#                       # dropoff 포함할 경우 아래 리스트에 'dropoff_datetime'도 추가
#                       parse_dates=['pickup_datetime'])
# taxi_df.columns = short_cols
# taxi_df.head()

,p_dt,pas,p_lng,p_lat,d_lng,d_lat,dur
0,4/30/16 23:59,1,-73.987793,40.724792,-73.975616,40.656445,1454
1,4/30/16 23:59,1,-73.957596,40.717770,-73.951424,40.775230,1409
2,4/30/16 23:59,2,-74.000954,40.742031,-73.947708,40.782200,1081
3,4/30/16 23:58,1,-73.985733,40.738258,-73.993179,40.754890,800
4,4/30/16 23:58,1,-74.006615,40.740650,-73.985619,40.723362,1151


In [4]:
# test.csv (날짜 대충 읽는 버전, 빠름)
test_df = pd.read_csv(dataLoc + testfile, usecols=cols_test)
test_df.columns = short_test
test_df.head()

# test.csv (날짜 제대로 읽어오는 버전, 오래걸림)
# test_df = pd.read_csv(dataLoc + testfile, \
#                       usecols=cols_test, \
#                       infer_datetime_format=True, \
#                       parse_dates=['pickup_datetime'])
# test_df.columns = short_test
# test_df.head()

,p_dt,pas,p_lng,p_lat,d_lng,d_lat
0,6/30/16 23:59,5,-73.998291,40.722618,-73.971779,40.762470
1,6/30/16 23:59,2,-73.955109,40.689564,-73.978203,40.685802
2,6/30/16 23:59,1,-73.978416,40.791576,-73.973160,40.675968
3,6/30/16 23:59,2,-73.873093,40.774097,-73.926704,40.856739
4,6/30/16 23:58,1,-73.979416,40.755211,-74.003548,40.738224


In [9]:
np.percentile(taxi_df['p_lat'], 0)

-78.547401429999994

In [11]:
taxi_df.shape

(701778, 7)

In [18]:
lower = np.percentile(taxi_df['p_lng'], 0.3)
upper = np.percentile(taxi_df['p_lng'], 99.7)
print(lower)
print(upper)

-74.01612854
-73.7767868


In [13]:
filtered_df = taxi_df[ (taxi_df['p_lng'] > lower) \
                     & (taxi_df['p_lng'] < upper)]
filtered_df.shape

(697482, 7)

In [34]:
filtered_df.head()

,p_dt,pas,p_lng,p_lat,d_lng,d_lat,dur
0,4/30/16 23:59,1,-73.987793,40.724792,-73.975616,40.656445,1454
1,4/30/16 23:59,1,-73.957596,40.717770,-73.951424,40.775230,1409
2,4/30/16 23:59,2,-74.000954,40.742031,-73.947708,40.782200,1081
3,4/30/16 23:58,1,-73.985733,40.738258,-73.993179,40.754890,800
4,4/30/16 23:58,1,-74.006615,40.740650,-73.985619,40.723362,1151


In [41]:
taxi_df.iloc[2].head()

p_dt     4/30/16 23:59
pas                  2
p_lng          -74.001
p_lat           40.742
d_lng         -73.9477
Name: 2, dtype: object

In [24]:
# p_lng is idx2
# ...
# d_lat is idx5

filtered_df = taxi_df

for i in range(2, 6):
    lower = np.percentile(taxi_df.iloc[:, i], 0.3)
    upper = np.percentile(taxi_df.iloc[:, i], 99.7)
    filtered_df = filtered_df[ (filtered_df.iloc[:, i] > lower) \
                         & (filtered_df.iloc[:, i] < upper) ]

filtered_df.shape
    

(686750, 7)

In [29]:
for col in short_cols[2:6]:
    print(col)
    print(np.percentile(filtered_df[col], 0) - np.percentile(taxi_df[col], 0.3) > 0)
    print(np.percentile(taxi_df[col], 99.7) - np.percentile(filtered_df[col], 100) > 0)
    print("filt min", col, np.percentile(filtered_df[col], 0))
    print("orig min", col, np.percentile(taxi_df[col], 0.3))
    print("filt max", col, np.percentile(filtered_df[col], 100))
    print("orig max", col, np.percentile(taxi_df[col], 99.7))
    

p_lng
True
True
filt min p_lng -74.01612091
orig min p_lng -74.01612854
filt max p_lng -73.77679443
orig max p_lng -73.7767868
p_lat
True
True
filt min p_lat 40.64365005
orig min p_lat 40.64364624
filt max p_lat 40.82314301
orig max p_lat 40.8231481078
d_lng
True
True
filt min d_lng -74.03305054
orig min d_lng -74.0331471945
filt max d_lng -73.77869415
orig max d_lng -73.7786788366
d_lat
True
True
filt min d_lat 40.63060379
orig min d_lat 40.6305961366
filt max d_lat 40.86594009
orig max d_lat 40.86598587


In [32]:
filtered_df.shape

(686750, 7)

In [34]:
filtered_df.to_csv('../_data/train_03_997.csv')

In [38]:
filt_test = pd.read_csv('../_data/train_03_997.csv',\
                        infer_datetime_format=True, \
                        parse_dates=['p_dt'])

filt_test.head()

,Unnamed: 0,p_dt,pas,p_lng,p_lat,d_lng,d_lat,dur
0,0,2016-04-30 23:59:00,1,-73.987793,40.724792,-73.975616,40.656445,1454
1,1,2016-04-30 23:59:00,1,-73.957596,40.717770,-73.951424,40.775230,1409
2,2,2016-04-30 23:59:00,2,-74.000954,40.742031,-73.947708,40.782200,1081
3,3,2016-04-30 23:58:00,1,-73.985733,40.738258,-73.993179,40.754890,800
4,4,2016-04-30 23:58:00,1,-74.006615,40.740650,-73.985619,40.723362,1151


In [42]:
del filt_test['Unnamed: 0']

In [46]:
filt_test.head()

,p_dt,pas,p_lng,p_lat,d_lng,d_lat,dur
0,2016-04-30 23:59:00,1,-73.987793,40.724792,-73.975616,40.656445,1454
1,2016-04-30 23:59:00,1,-73.957596,40.717770,-73.951424,40.775230,1409
2,2016-04-30 23:59:00,2,-74.000954,40.742031,-73.947708,40.782200,1081
3,2016-04-30 23:58:00,1,-73.985733,40.738258,-73.993179,40.754890,800
4,2016-04-30 23:58:00,1,-74.006615,40.740650,-73.985619,40.723362,1151


In [47]:
filt_test.to_csv('../_data/train_03_997_dates.csv', index=False)

In [50]:
new_filt = pd.read_csv('../_data/train_03_997_dates.csv')

new_filt.head()

,p_dt,pas,p_lng,p_lat,d_lng,d_lat,dur
0,2016-04-30 23:59:00,1,-73.987793,40.724792,-73.975616,40.656445,1454
1,2016-04-30 23:59:00,1,-73.957596,40.717770,-73.951424,40.775230,1409
2,2016-04-30 23:59:00,2,-74.000954,40.742031,-73.947708,40.782200,1081
3,2016-04-30 23:58:00,1,-73.985733,40.738258,-73.993179,40.754890,800
4,2016-04-30 23:58:00,1,-74.006615,40.740650,-73.985619,40.723362,1151


In [56]:
new_filt['p_dt'] = pd.to_datetime(new_filt['p_dt'])

In [58]:
filtered_df['p_dt'] = pd.to_datetime(filtered_df['p_dt'])

In [62]:
new_filt['p_dt'].dt.dayofweek.head()

0    5
1    5
2    5
3    5
4    5
Name: p_dt, dtype: int64